In [1]:
%matplotlib inline
import matplotlib.pylab as plt

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np


hrs=3
candlestick_frame = 4*hrs
pnl_threshold = 3
df = pd.read_csv('../../data/BTCUSDT.csv')

ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
tot = len(ts)
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(open[idx+1:idx+candlestick_frame])
        min_low = min(open[idx+1:idx+candlestick_frame])
        entry_price = open[idx+1]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     


long=[]
short=[]
dont_trade=[]
signal = []

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        if long_runup_lst[idx] >= pnl_threshold:
            signal.append(1)
        elif short_runup_lst[idx] >= pnl_threshold:
            signal.append(2)
        else:
            signal.append(0)
    else:
        signal.append(0)


pc = [0] + [round((open[i]*100/open[(i-1)])-100, 4) for i in range(1,len(open), 1)]
df = pd.DataFrame()
df['0'] = pc
y = pd.Series( (i for i in signal) )

In [2]:
df["id"] = df.index
df = df.melt(id_vars="id", var_name="time").sort_values(["id", "time"]).reset_index(drop=True)

In [3]:
df.head()


,id,time,value
0,0,0,0.0
1,1,0,0.0
2,2,0,0.0
3,3,0,0.0
4,4,0,0.0


In [4]:
X = extract_features(df[df["id"] < len(df)], column_id="id", column_sort="time", impute_function=impute)
X.head()


Feature Extraction:  70%|███████   | 28/40 [09:51<03:40, 18.39s/it]  

: 

116171